<a href="https://colab.research.google.com/github/LauraPiv/CNN-DogXWolf/blob/main/CNN_DOGxWOLF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Imports**

In [ ]:
!pip install kaggle

In [6]:
import numpy as np
import pandas as pd
import os
import random
import matplotlib.pyplot as plt
import keras.backend as K
from keras.models import Sequential
from keras.utils import to_categorical
from tensorflow.keras.layers import BatchNormalization
from keras.layers import Dense, Flatten, Convolution2D, Conv2D, MaxPooling2D,BatchNormalization, Dropout, Activation, AveragePooling2D
from keras.preprocessing import image
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
import seaborn as sns
from PIL import Image
import tensorflow as ts
from sklearn.metrics import confusion_matrix, classification_report
import kagglehub
import requests
from io import BytesIO

#**Reading Data**

In [ ]:
from google.colab import files
uploaded = files.upload() #Go to your Kaggle account and create an API token, then upload the saved file from your computer.

kaggle_dir = os.path.expanduser("~/.kaggle")
if not os.path.exists(kaggle_dir):
    os.makedirs(kaggle_dir)

kaggle_file_name = next(iter(uploaded))
shutil.move(kaggle_file_name, os.path.join(kaggle_dir, "kaggle.json"))

os.chmod(os.path.join(kaggle_dir, "kaggle.json"), 0o600)

!kaggle datasets download -d harishvutukuri/dogs-vs-wolves

!unzip -o dogs-vs-wolves.zip


Saving kaggle (1).json to kaggle (1) (2).json
Dataset URL: https://www.kaggle.com/datasets/harishvutukuri/dogs-vs-wolves
License(s): DbCL-1.0
 95% 210M/222M [00:01<00:00, 119MB/s] 
100% 222M/222M [00:02<00:00, 114MB/s]
Archive:  dogs-vs-wolves.zip
  inflating: data/dogs/n02085620_13964.jpg  
  inflating: data/dogs/n02085620_1617.jpg  
  inflating: data/dogs/n02085620_2204.jpg  
  inflating: data/dogs/n02085620_2887.jpg  
  inflating: data/dogs/n02085620_3409.jpg  
  inflating: data/dogs/n02085620_3423.jpg  
  inflating: data/dogs/n02085620_4207.jpg  
  inflating: data/dogs/n02085620_588.jpg  
  inflating: data/dogs/n02085620_8637.jpg  
  inflating: data/dogs/n02085782_1600.jpg  
  inflating: data/dogs/n02085782_1764.jpg  
  inflating: data/dogs/n02085782_2269.jpg  
  inflating: data/dogs/n02085782_3021.jpg  
  inflating: data/dogs/n02085782_3899.jpg  
  inflating: data/dogs/n02085936_10148.jpg  
  inflating: data/dogs/n02085936_10297.jpg  
  inflating: data/dogs/n02085936_10307.jpg  
 

In [ ]:
path = kagglehub.dataset_download("harishvutukuri/dogs-vs-wolves")

print("Path to dataset files:", path)

100%|██████████| 222M/222M [00:04<00:00, 51.7MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/harishvutukuri/dogs-vs-wolves/versions/2


In [ ]:
data_path = '/root/.cache/kagglehub/datasets/harishvutukuri/dogs-vs-wolves/versions/2/data'
print("Directory content 'data':", os.listdir(data_path))

Conteúdo do diretório 'data': ['dogs', 'wolves']


In [ ]:
import os
dogs = []
wolves = []
img_size = 300
DOGS_IMGS_PATH = os.path.join(data_path, 'dogs')
WOLVES_IMGS_PATH = os.path.join(data_path, 'wolves')
DIRS = [(0, DOGS_IMGS_PATH), (1, WOLVES_IMGS_PATH)]

# **Training Data and Show**

In [ ]:
train_images = []
labels = []
for num, _dir in DIRS:
    _dir = _dir + '/'
    count = 0
    for file in os.listdir(_dir):
        if count >= 1000:
            break
        img = image.load_img(_dir + str(file), target_size=(img_size, img_size))
        img = image.img_to_array(img)
        img = img/255
        train_images.append(img)
        labels.append(num)
        count += 1

In [ ]:
train_images[1].shape


(300, 300, 3)

In [ ]:
len(train_images)


2000

In [ ]:
X = np.array(train_images)

# **Training & Testing data for Work**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.1, random_state=101)

In [ ]:
len(X_train)

1800

In [ ]:
y_train_labels = to_categorical(y_train)

# **Initialize the model**

In [ ]:
def build(width, height, depth, classes):

    model = Sequential()
    inputShape = (height, width, depth)
    chanDim = -1

    if K.image_data_format() == 'channels_first':
        inputShape = (depth, height, width)
        chanDim = 1

    #Input layer
    model.add(Conv2D(32, (3, 3), padding='same', input_shape=inputShape))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    # (CONV -> RELU -> POOL)
    model.add(Conv2D(64, (3,3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    # (CONV -> RELU -> POOL)
    model.add(Conv2D(128, (3,3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    # Flatten -> Dense -> RELU
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation("relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

    # Camada final
    model.add(Dense(classes))
    model.add(Activation('sigmoid'))

    return model

In [ ]:
model = build(img_size,img_size, 3, 2)

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_1 (Conv2D)                    │ (None, 300, 300, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation (Activation)              │ (None, 300, 300, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 300, 300, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 150, 150, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 150, 150, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 150, 150, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_1 (Activation)            │ (None, 150, 150, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 150, 150, 64)        │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 75, 75, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 75, 75, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 75, 75, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_2 (Activation)            │ (None, 75, 75, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 75, 75, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 37, 37, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 37, 37, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 175232)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │      89,719,296 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_3 (Activation)            │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 89,816,514 (342.62 MB)

 Trainable params: 89,815,042 (342.62 MB)

 Non-trainable params: 1,472 (5.75 KB)

In [ ]:
history = model.fit(X_train, y_train_labels, batch_size=32, epochs=15, validation_split=0.1)

NameError: name 'model' is not defined

#**Classifying New Photos**



In [ ]:
input_shape = (300, 300, 3)

def load_and_preprocess_image(img_path_or_url):
    try:

        if img_path_or_url.startswith('http'):
            response = requests.get(img_path_or_url)
            img = Image.open(BytesIO(response.content)).convert('RGB')
        else:
            img = Image.open(img_path_or_url).convert('RGB')

        img = img.resize((input_shape[1], input_shape[0]))
        img_array = image.img_to_array(img)

        img_array = np.expand_dims(img_array, axis=0)

        img_array = img_array / 255.0

        return img_array
    except Exception as e:
        print(f"Erro ao carregar a imagem: {e}")
        return None

def predict_image(img_path_or_url):
    img_array = load_and_preprocess_image(img_path_or_url)

    if img_array is None:
        return

    #Prediction
    try:
        prediction = model.predict(img_array)

        dog_confidence = prediction[0][0] * 100
        wolf_confidence = (1 - prediction[0][0]) * 100

        predicted_class = 0 if dog_confidence > wolf_confidence else 1
        label = 'dog' if predicted_class == 0 else 'wolf'

        plt.imshow(img_array[0])
        plt.title(f"Predicted: {label}\nDog: {dog_confidence:.2f}%, Wolf: {wolf_confidence:.2f}%")
        plt.axis('off')
        plt.show()

    except Exception as e:
        print(f"Erro ao fazer a previsão: {e}")

#Example:
img_path = '/kaggle/input/testes/cao3.jfif'  #Replace with the path or URL of the image you want to test
predict_image(img_path)
